In [ ]:
import os
os.environ["VRE_LOGLEVEL"] = "0"
from pathlib import Path
from pprint import pprint
import subprocess
from omegaconf import OmegaConf
from vre.representations import build_representations_from_cfg, ReprOut, Representation
from vre.readers.multitask_dataset import MultiTaskDataset, MultiTaskItem
from vre.utils import reorder_dict, collage_fn, image_add_title, get_project_root, MemoryData
from vre_repository import get_vre_repository

import numpy as np
import torch as tr
import matplotlib.pyplot as plt

from semantic_mapper import plot_one

%load_ext autoreload
%autoreload 2
%matplotlib inline

#### Define the data path.
It can be a normal video (mp4) or a previous export of VRE if it contains the 6 relevant 'seed' experts:
- rgb
- opticalflow_rife
- depth_marigold
- normals_svd(depth_marigold)
- semantic_mask2former_mapillary_49189528_0
- semantic_mask2former_coco_47429163_0

You can also generate it using `vre /path/to/video.mp4 -o out_dir --config_path cfg.yaml`

In [ ]:
cfg_path = Path.cwd() / "cfg.yaml"
# data_path = get_project_root() / "resources/test_video.mp4" # It can also be a previously exported VRE dir
data_path = Path.cwd() / "data"
vre_dir = data_path
if data_path.suffix == ".mp4":
    vre_dir = Path.cwd() / f"data_{data_path.name}"
    frames = ["1", "100", "1000", "3000"]
    if not vre_dir.exists():
        subprocess.run(args=["vre", str(data_path), "--config_path", str(cfg_path),
                             "-o", str(vre_dir), "--frames", *frames],
                        env={**os.environ.copy(), **{"VRE_DEVICE": "cuda" if tr.cuda.is_available() else "cpu"}})
    else:
        print(f"Out dir '{vre_dir}' exists. Remove it first")
else:
    print("Not a video, but a dir of exported VRE results. Skipping")

#### Instantiate the MultiTaskDataset object

This is only done for the initial seed expert tasks defined earlier. We'll also plot them.

In [ ]:
task_names = ["rgb", "depth_marigold", "normals_svd(depth_marigold)", "opticalflow_rife",
              "semantic_mask2former_coco_47429163_0", "semantic_mask2former_mapillary_49189528_0"]
order = ["rgb", "semantic_mask2former_mapillary_49189528_0", "semantic_mask2former_coco_47429163_0",
            "depth_marigold", "normals_svd(depth_marigold)"]

representations = build_representations_from_cfg(cfg_path, get_vre_repository())
name_to_repr = {r.name: r for r in representations}
reader = MultiTaskDataset(vre_dir, task_names=task_names,
                          task_types=name_to_repr, handle_missing_data="fill_nan",
                          normalization="min_max", cache_task_stats=True, batch_size_stats=100)
orig_task_names = list(reader.task_types.keys())

print(reader)
print("== Shapes ==")
pprint(reader.data_shape)

data, name = reader[np.random.randint(0, len(reader))]
collage = plot_one(data, title=name, order=order, name_to_task=reader.name_to_task)
plt.figure(figsize=(15, 5))
plt.imshow(collage)
plt.show()

#### Add the newly defined tasks

Thery are read from `semantic_mapper.py` but you can define new ones here too using the primitives from there.
Each of them is added to the MultiTaskDataset object one by one.

The computation in the reader is done on the fly, meaning they are derived from the underlying existing experts
on the fly.

In [ ]:
from semantic_mapper import get_new_semantic_mapped_tasks # put this here for auto loading
new_tasks = get_new_semantic_mapped_tasks(include_semantic_output=False)
for task_name in reader.task_names:
    if task_name not in orig_task_names:
        reader.remove_task(task_name)
for new_task in new_tasks.values():
    reader.add_task(new_task, overwrite=True)

print("== Random loaded item ==")
ixs = np.random.permutation(range(len(reader))).tolist()
for ix in ixs:
    data, name = reader[ix]
    print(data)
    plt.figure(figsize=(20, 10))
    plt.imshow(collage := plot_one(data, title=name, order=order, name_to_task=reader.name_to_task))
    plt.show()
    break